# Capstone Project Notebook
### by Dimitar Kunev
This notebook will be used as the primary vehicle for my capstone project.

In [2]:
# set up the libraries
#    this list will grow as I progress through the project
import pandas as pd
import numpy as np


In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
